# TradingClient example

## Prepare client

In [ ]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [ ]:
# import the package
from osmosispy.trading_client import BinanceClient, ITradingClient
import time

don't forget to set your API key and secret to the environment variables
`BINANCE_API_KEY` and `BINANCE_API_SECRET` respectively.

```bash
export BINANCE_API_KEY=qwerty
export BINANCE_API_SECRET=asdfgh
```

In [ ]:
# create a client from the environment variables
client = BinanceClient.from_env()

Alternatively, you can pass them to the client constructor.

In [ ]:
# create a client from the constructor
client: ITradingClient = BinanceClient(
    api_key="qwerty",
    api_secret="asdfgh",
)

## Get osmosis trading info

In [ ]:
fee = client.get_trading_fee("USDT")
# fee object will contain the following fields:
# maker_commision: float
# taker_commision: float

print(f"Trading fee: {fee}")

In [ ]:
price_in_usdt = client.get_price(price_in_symbol = "USDT")

print(f"Current price in USDT:\n{price_in_usdt}")

## Listen trades

In [ ]:
# create trade callback
def trade_callback(msg):
    print(f"trade: {msg}")

# start listening to trades
client.listen_trades(cb=trade_callback)

In [ ]:
# listen for 60 seconds
time.sleep(60)

In [ ]:
# stop listening to trades
client.stop_ws()